# チャットモデルとプロンプトテンプレートを使用してシンプルなLLMアプリケーションを構築する

[Build a simple LLM application with chat models and prompt templates \| 🦜️🔗 LangChain](https://python.langchain.com/docs/tutorials/llm_chain/)

このクイックスタートでは、LangChainを使用してシンプルなLLMアプリケーションを構築する方法を紹介します。このアプリケーションは、英語のテキストを他の言語に翻訳します。これは比較的シンプルなLLMアプリケーションで、単一のLLM呼び出しといくつかのプロンプトだけです。それでも、LangChainを始めるには素晴らしい方法です。いくつかのプロンプトとLLM呼び出しだけで多くの機能を構築できます！

このチュートリアルを読んだ後、次のことについての概要を理解できます：

- [言語モデル](https://python.langchain.com/docs/concepts/chat_models/)の使用
- [プロンプトテンプレート](https://python.langchain.com/docs/concepts/prompt_templates/)の使用
- [LangSmith](https://docs.smith.langchain.com/)を使用したアプリケーションのデバッグとトレース

さあ、始めましょう！

In [0]:
%pip install -qU langchain[openai] databricks-langchain mlflow
%restart_python

In [0]:
import mlflow

# MLflow Tracingの有効化
mlflow.langchain.autolog()

In [0]:
import os
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("demo-token-takaaki.yayoi", "openai_api_key")

## 言語モデルの利用

まず、言語モデルを単独で使用する方法を学びましょう。LangChainは、多くの異なる言語モデルをサポートしており、それらを相互に交換して使用できます。特定のモデルの使用を開始するための詳細については、[サポートされているインテグレーション](https://python.langchain.com/docs/integrations/chat/)を参照してください。

In [0]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

まず、モデルを直接使用してみましょう。[ChatModels](https://python.langchain.com/docs/concepts/chat_models/)はLangChain [Runnables](https://python.langchain.com/docs/concepts/runnables/)のインスタンスであり、それらと対話するための標準インターフェースを提供します。モデルを単純に呼び出すには、[メッセージ](https://python.langchain.com/docs/concepts/messages/)のリストを`.invoke`メソッドに渡すことができます。

In [0]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("以下を英語から日本語に翻訳して"),
    HumanMessage("hi!"),
]

model.invoke(messages)

**APIリファレンス:** [HumanMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html) | [SystemMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html)

ChatModelsは[メッセージ](https://python.langchain.com/docs/concepts/messages/)オブジェクトを入力として受け取り、メッセージオブジェクトを出力として生成します。メッセージオブジェクトはテキストコンテンツに加えて、会話の[役割](https://python.langchain.com/docs/concepts/messages/#role)を伝え、[ツールの呼び出し](https://python.langchain.com/docs/concepts/tool_calling/)やトークン使用量などの重要なデータを保持します。

LangChainは、文字列や[OpenAI形式](https://python.langchain.com/docs/concepts/messages/#openai-format)でのチャットモデル入力もサポートしています。以下は等価です：

In [0]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

## ストリーミング

チャットモデルは[Runnable](https://python.langchain.com/docs/concepts/runnables/)であるため、非同期およびストリーミングモードの呼び出しを含む標準インターフェースを公開しています。これにより、チャットモデルから個々のトークンをストリーミングすることができます：

In [0]:
for token in model.stream(messages):
    print(token.content, end="|")

ストリーミングチャットモデルの出力に関する詳細は、[このガイド](https://python.langchain.com/docs/how_to/chat_streaming/)で確認できます。

## プロンプトテンプレート
現在、メッセージのリストを直接言語モデルに渡しています。このメッセージのリストはどこから来るのでしょうか？通常、これはユーザー入力とアプリケーションロジックの組み合わせから構築されます。このアプリケーションロジックは通常、生のユーザー入力を取り、それを言語モデルに渡す準備ができたメッセージのリストに変換します。一般的な変換には、システムメッセージの追加やユーザー入力を使用したテンプレートのフォーマットが含まれます。

[プロンプトテンプレート](https://python.langchain.com/docs/concepts/prompt_templates/)は、この変換を支援するためにLangChainで設計された概念です。これらは生のユーザー入力を受け取り、言語モデルに渡す準備ができたデータ（プロンプト）を返します。

ここでプロンプトテンプレートを作成しましょう。これは2つのユーザー変数を受け取ります：

- `language`: 翻訳する言語
- `text`: 翻訳するテキスト

In [0]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "以下を英語から {language} に翻訳して"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

**APIリファレンス:** [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

ChatPromptTemplateは、単一のテンプレートで複数の[メッセージロール](https://python.langchain.com/docs/concepts/messages/#role)をサポートしていることに注意してください。システムメッセージに`language`パラメータをフォーマットし、ユーザーメッセージにユーザーのテキストをフォーマットします。

このプロンプトテンプレートへの入力はディクショナリーです。このプロンプトテンプレートを単独で操作して、何ができるかを確認できます。

In [0]:
prompt = prompt_template.invoke({"language": "日本語", "text": "hi!"})

prompt

2つのメッセージで構成される`ChatPromptValue`が返されることがわかります。メッセージに直接アクセスしたい場合は、次のようにします:

In [0]:
prompt.to_messages()

最後に、フォーマットされたプロンプトでチャットモデルを呼び出すことができます:

In [0]:
response = model.invoke(prompt)
print(response.content)

## まとめ

以上です！このチュートリアルでは、最初のシンプルなLLMアプリケーションの作成方法を学びました。言語モデルの操作方法、プロンプトテンプレートの作成方法、LangSmithを使用して作成したアプリケーションの優れた可観測性を得る方法を学びました。

これは、熟練したAIエンジニアになるために学ぶべきことのほんの一部に過ぎません。幸いなことに、他にも多くのリソースがあります！

LangChainのコアコンセプトについてさらに読みたい場合は、詳細な[コンセプトガイド](https://python.langchain.com/docs/concepts/)があります。

これらのコンセプトに関して具体的な質問がある場合は、ハウツーガイドの以下のセクションをチェックしてください:

- [Chat models](https://python.langchain.com/docs/how_to/#chat-models)
- [Prompt templates](https://python.langchain.com/docs/how_to/#prompt-templates)